# Imports

In [1]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
import autograd.numpy.random as npr
import myOptimizers as optims
import os
import importlib

# Function Definitions

In [2]:
# Weight initialisation
def init_random_params(scale, layer_sizes, rs=npr.RandomState(42)):
    return [(rs.randn(insize, outsize) * scale,   # weight matrix
             rs.randn(outsize) * scale)           # bias vector
            for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]

# Activation function
def tanh(x):
    return (2 / (1.0 + np.exp(-2*x))) - 1
    #return np.log(1+np.exp(x))
    #return np.exp(x)

# Returns the NN output AFTER enforcing the boundary condition. This is the wavefunction (wf).
def psi(nnparams, inputs):
    origInput = inputs
    xTilde = (origInput - x0) / (x1 - x0)
    
    for W, b in nnparams:
        outputs = np.dot(inputs, W) + b
        inputs = tanh(outputs)    
        
    # Enforcing the boundary conditions
    return (1-xTilde)*psi0 + xTilde*psi1 + (1-np.exp(xTilde * (1 - xTilde))) * outputs


dpsi = elementwise_grad(psi, 1) # Function for returning first derivative of wf (not used explicitly)
ddpsi = elementwise_grad(dpsi, 1) # Second derivative of wf. 

In [3]:
PRC1 = 20 # Probability regularisation co-efficient for degree 1 (and -1)
PRC2 = PRC1/(2*2) # Probability regularisation co-efficient for degree 2 (and -2)
PRC3 = PRC2/(2*3) # Probability regularisation co-efficient for degree 3 (and -3)
PRC4 = PRC3/(2*4) # Probability regularisation co-efficient for degree 4 (and -4)


def floatStr (Efloat) :
    Eint = int(np.floor(Efloat))
    if Efloat - Eint == 0 :
        return str(int(np.floor(Efloat)))
    else :
        Estr = str(Efloat)
        return str(int(np.floor(Efloat))) + "p" + Estr[Estr.find('.')+1:]

# Objective function
def objective(params, step):
    nnparams = params['nn']
    E = params['E']        
    
    wf = psi(nnparams,x)    # Wavefunction
    diffeq = -0.5 * ddpsi(nnparams, x)  - E*wf  # Schrodinger equation in functional form
    y2 = wf**2   # Probability density function
    
    # This is a numerical trapezoid integration
    prob = np.sum((y2[1:] + y2[0:-1]) / 2 * (x[1:] - x[0:-1]))

    
    #1/(10**8.5)*(1/(E-1.5)**4 + 1/(E-3)**4)
    return np.mean(diffeq**2) + (1-prob)**2 + (PRC1/prob + PRC1*prob) + \
        (PRC2/prob**2 + PRC2*prob**2) + (PRC3/prob**3 + PRC3*prob**3) + \
        (PRC4/prob**4 + PRC4*prob**4) +1/(10**EregCoeffExp)*(1/(E-E1)**EregN + 1/(E-E2)**EregN)


trunc = -1
iterList = []
EregList = []
EregDerList = []
diffeqList = []
diffeqDiffList = []
probList = []
lossList = []
EList = []
# Helper function for callback. Prints the following -
# 1. Iteration no.
# 2. MSE term
# 3. Probability integral
# 4. Loss
# 5. Eigen-energy 
def objectiveDebug (params, step) :
    nnparams = params['nn']
    E = params['E']   
    
    wf = psi(nnparams,x)    # Wavefunction
    diffeq = -0.5 * ddpsi(nnparams, x)  - E*wf  # Schrodinger equation in functional form
    y2 = wf**2   # Probability density function
    mean = np.mean(diffeq**2)  # MSE

    # This is a numerical trapezoid integration
    prob = np.sum((y2[1:] + y2[0:-1]) / 2 * (x[1:] - x[0:-1]))
    Ereg = 1/(10**8.5)*(1/(E-1.5)**4 + 1/(E-3)**4)
    #EregDer = -4/(10**8.5)*(1/(E-1.5)**5 + 1/(E-3)**5)
    EregDer = -EregN/(10**EregCoeffExp)*(1/(E-E1)**(EregN+1) + 1/(E-E2)**EregN)
    
    

    loss = mean + (1-prob)**2 + (PRC1/prob + PRC1*prob) + \
        (PRC2/prob**2 + PRC2*prob**2) + (PRC3/prob**3 + PRC3*prob**3) + \
        (PRC4/prob**4 + PRC4*prob**4) + Ereg
    
    iterList.append (step)
    diffeqList.append (mean)
    probList.append (prob)
    lossList.append (loss)
    EregList.append (Ereg)
    EregDerList.append (EregDer)
    EList.append (float(params['E']))

    print ("Iteration " + str(step) + "\ndiffeq = " + str(mean) + " prob = " + str(prob) + \
           " Ereg = " + str(Ereg))
    print ("loss = " + str(loss) + " E = " + str(params['E']))
    print ("EregDer = " + str(EregDer))

# Callback function.
def callback(params, step, g):
    global trunc
    global diffeqList
    
    if step % 100 == 0:
        objectiveDebug (params, step)
        
        if step > 100 :
            if trunc == -1 and np.abs(diffeqDiffList[-1]) < 10**-2 and diffeqDiffList[-1] < 0 :
                trunc = int(step/100)
                print ("Set at index = " + str(trunc))        

# Initialisation

In [4]:
psi0 = 0 # Value of first boundary
psi1 = 0 # Value of second boundary
L = 6 # Length of the well
scale = 2*L # Scale for initialisation
divs = 300 # divisions of the input space

x0 = 0  # Position of first boundary
x1 = L  # Position of second boundary

In [5]:
# Initialising the weights. They usually fall between 0 and 1 which causes the initial wavefunction
# to have a small probability integral over the range. Hence, I've put a scale factor of 2 that changes gives it
# a random (but sizeable) shape so that over the epochs, the probability integral converges to 1.
nnparams = init_random_params(scale/(x1-x0), layer_sizes=[1, 32, 32, 1])

# Intialise a guess eigenvalue to find a true eigenvalue near it
initE = 0.6
params = {'nn': nnparams, 'E': initE}
E1 = 0.4
E2 = 1.0
EregN = 10
#EregCoeffExp = (2 - int(np.ceil(np.log10(E2-E1)))) * (EregN + 3)
EregCoeffExp = (2 - np.log10(E2-E1)) * (EregN + 3)

# Input space
x = np.linspace(x0, x1, divs)[:, None]

# Setting up the name for saving files
name = "L" + floatStr(L) + "E" + floatStr(E1) + "E" + floatStr(initE) + "E" + floatStr(E2)

# Optimization

In [6]:
optims = importlib.reload (optims)
params = optims.myAdam (grad(objective), params, callback=callback, step_size=0.001, diffeqList=diffeqList, probList=probList, diffeqDiffList = diffeqDiffList)

Iteration 100
diffeq = 1.5244435082160785 prob = 1.0036069014873175 Ereg = 7.148522929269887e-09
loss = 53.40009387375528 E = 0.6813082691073031
EregDer = 7.017788864126922e-18
---------------------------------
Iteration 200
diffeq = 0.68599816235184 prob = 1.0009451328498356 Ereg = 9.885280414388397e-09
loss = 52.56104294346081 E = 0.7455928651143434
EregDer = 7.959416209077349e-16
diffeqDiff = -0.8384453458642386
---------------------------------
Iteration 300
diffeq = 0.4297417642358702 prob = 1.0026185970854997 Ereg = 1.376242763479942e-08
loss = 52.30508489341637 E = 0.8059222409808736
EregDer = 2.341559000331198e-13
diffeqDiff = -0.2562563981159698
---------------------------------
Iteration 400
diffeq = 0.297178301929893 prob = 1.0013755095180197 Ereg = 1.8968193358056626e-08
loss = 52.17227310982017 E = 0.8597352522938301
EregDer = 2.143270656065793e-10
diffeqDiff = -0.1325634623059772
---------------------------------
Iteration 500
diffeq = 0.21102857367989933 prob = 1.0008833

Iteration 3300
diffeq = 0.024744075475852424 prob = 0.9998081717850519 Ereg = 3.247819176721169e-08
loss = 51.899745954337405 E = 0.9406400074772778
EregDer = 0.014915082563898781
diffeqDiff = -0.000511652384785611
brkCount3 = 8
---------------------------------
Iteration 3400
diffeq = 0.024277635599270073 prob = 0.99980247918708 Ereg = 3.240835134364208e-08
loss = 51.89927962561259 E = 0.9403377004767399
EregDer = 0.013405916230841822
diffeqDiff = -0.0004664398765823509
brkCount3 = 9
---------------------------------
Iteration 3500
diffeq = 0.0238534825760376 prob = 0.9997972921254039 Ereg = 3.2337576507857727e-08
loss = 51.89885557669578 E = 0.9400305145355758
EregDer = 0.012035370811853862
diffeqDiff = -0.000424153023232475
brkCount3 = 10
---------------------------------
Iteration 3600
diffeq = 0.023468659627920416 prob = 0.9997926058468221 Ereg = 3.226646383821372e-08
loss = 51.89847085011856 E = 0.9397210122478395
EregDer = 0.010802176909144623
diffeqDiff = -0.0003848229481171819

KeyboardInterrupt: 

# Plots

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
x = np.linspace(x0, x1)[:, None]
y = psi(params['nn'], x)

plt.plot(x, y, label='NN')  # Plotting the NN output
plt.plot (x , np.sqrt(2/L) * np.sin(3*np.pi*x/L), label='True')
plt.legend()
plt.savefig (name + "_plot.png")

In [ ]:
fig = plt.figure (figsize=(10,10))
plt.clf ()

plt.subplot (3 , 2 , 1)
plt.plot (iterList, diffeqList)
plt.xlabel ('iterations')
plt.ylabel ('diffeq')

plt.subplot (3, 2, 2)
plt.plot (iterList, probList)
plt.xlabel ('iterations')
plt.ylabel ('prob')

plt.subplot (3, 2, 3)
plt.plot (iterList, lossList)
plt.xlabel ('iterations')
plt.ylabel ('loss')

plt.subplot (3, 2, 4)
plt.plot (iterList, EList)
plt.ylabel ('Eigenenergy')
plt.xlabel ('iterations')
plt.tight_layout()

plt.subplot (3, 2, 5)
plt.plot (iterList, EregList)
plt.ylabel ('EigReg')
plt.xlabel ('iterations')
plt.tight_layout()

plt.subplot (3, 2, 6)
plt.plot (iterList, EregDerList)
plt.ylabel ('EigRegDer')
plt.xlabel ('iterations')
plt.tight_layout()


plt.savefig (name + "_diagAll.png")

In [ ]:
if trunc != -1 :
    fig = plt.figure (figsize=(10,10))

    plt.subplot (3 , 2 , 1)
    plt.plot (iterList[trunc:], diffeqList[trunc:])
    plt.xlabel ('iterations')
    plt.ylabel ('diffeq')

    plt.subplot (3, 2, 2)
    plt.plot (iterList[trunc:], probList[trunc:])
    plt.xlabel ('iterations')
    plt.ylabel ('prob')

    plt.subplot (3, 2, 3)
    plt.plot (iterList[trunc:], lossList[trunc:])
    plt.xlabel ('iterations')
    plt.ylabel ('loss')

    plt.subplot (3, 2, 4)
    plt.plot (iterList[trunc:], EList[trunc:])
    plt.ylabel ('Eigenenergy')
    plt.xlabel ('iterations')
    plt.tight_layout()
    
    plt.subplot (3, 2, 5)
    plt.plot (iterList[trunc:], EregList[trunc:])
    plt.ylabel ('Ereg')
    plt.xlabel ('iterations')
    plt.tight_layout()
    

    plt.subplot (3, 2, 6)
    plt.plot (iterList[trunc:], EregDerList[trunc:])
    plt.ylabel ('EigRegDer')
    plt.xlabel ('iterations')
    plt.tight_layout()


    plt.savefig (name + "_diagTrunc" + str(trunc) + ".png")

In [ ]:
    fig = plt.figure (figsize=(5,5))
    plt.clf ()

    if trunc != -1 :
        plt.subplot (2 , 1 , 1)
        plt.plot (iterList, diffeqDiffList)
        plt.xlabel ('iterations')
        plt.ylabel ('diffeqDiff')

        plt.subplot (2 , 1 , 2)
        plt.plot (iterList[trunc:], diffeqDiffList[trunc:])
        plt.xlabel ('iterations')
        plt.ylabel ('diffeqDiff')
    else :
        plt.plot (iterList, diffeqDiffList)
        #plt.yticks ([min(diffeqDiffList), -10**-4, 10**-4, max(diffeqDiffList)])
        plt.xlabel ('iterations')
        plt.ylabel ('diffeqDiff')

    plt.tight_layout()
    plt.savefig (name + "_diffeqDiff.png")

# Write to file

In [ ]:
outfile = open(name + "_info.txt" , "w")

fstr = "diffeq = " + str(round(diffeqList[-1],8)) + \
"\nprob = " + str(round(probList[-1],8)) + \
"\nloss = " + str(round(lossList[-1],8)) + \
"\neigE = " + str(round(EList[-1],8)) + \
"\ndiffeqDiff = " + str(round(diffeqDiffList[-1],8))

    
outfile.write (fstr)
outfile.close ()